<h1><center>CSEN1022:Assignment 3</center></h1>
<h3><center>Winter 2021</center></h3>
<hr style="border:2px solid black"> </hr>

## <u> Please don't forget to fill in this data </u>
**Member 1**

Name:

GUC-ID:

Elective Tutorial No.:

**Member 2**

Name:

GUC-ID:

Elective Tutorial No.:

<hr style="border:2px solid black"> </hr>

## Imports (Don't Edit)

In [ ]:
# ONLY USE THESE IMPORTS.
# PLEASE DON'T EDIT THIS CELL.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Read Data

In [ ]:
# X_train, X_test
# THE image categories are different.

<hr style="border:2px solid black"> </hr>

# Perform K means clustering for all 3 classes (Training Data).
### Return (memberships, centroids, dbi) --> (vector, matrix, scalar value).

In [ ]:
###
#CODE HERE#
###

# return (memberships, centroids, dbi)

<hr style="border:2px solid black"> </hr>

# Repeat the previous process 10 times. 
### Pick the membership vector and the centroids matrix corresponding to the best dbi.
##### Make sure you return max_counts and confusion_matrix.
(keep history in whatever datastructure you like).

In [ ]:
#best_membership_matrix = 
#best_centroids = 
#best_dbi = 

###
#CODE HERE#
###

#max_counts = 
#confusion_matrix=

<hr style="border:2px solid black"> </hr>

## Don't Edit the Following Cells, Just Run & Save them.

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(['Airplane','Bird','Truck'],max_counts,'-o')
plt.title('Best Counts')

In [ ]:
plt.rc('figure', figsize=[5,5])
plt.matshow(confusion_matrix,cmap="Blues")
for i in range(0,confusion_matrix.shape[0]):
    for j in range(0,confusion_matrix.shape[1]):
        plt.annotate(confusion_matrix[i,j],(j,i))